# Segmenting and Clustering Toronto Assignment
This is part 2 of the assignment

<b>Inserting substantive code from first part of the assignment.</b>

In [1]:
import pandas as pd
import numpy as np
import requests
!pip install lxml
!pip install html5lib
!pip install beautifulsoup4
!pip install folium==0.5.0
import json
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
print ('Libraries imported.')
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df=pd.read_html(url)
Toronto_df=df[0]
Toronto_df.drop(Toronto_df[Toronto_df.Borough == 'Not assigned'].index, inplace=True)
Toronto_df.rename(columns={'Neighbourhood': 'Neighborhood'}, inplace=True)
Toronto_df2 = Toronto_df.groupby(['Postal Code', 'Borough'], sort=False)['Neighborhood'].apply(','.join).reset_index()
print("Final Shape: ",Toronto_df.shape)

Libraries imported.
Final Shape:  (103, 3)


<b> Per assignment instructions, attempting install and use of GeoCoder.</b>

In [3]:
!pip install geocoder
print('installed.')

     |████████████████████████████████| 98 kB 8.9 MB/s  eta 0:00:01
installed.


In [6]:
g = geocoder.google('Mountain View, CA')
g.latlng
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

NameError: name 'geocoder' is not defined

<b>Per assignment instructions, GeoCoder did indeed prove unreliable, and I will now utilize the provided link to the csv file.</b>
(https://cocl.us/Geospatial_data)

In [15]:
geo_coord_df = pd.read_csv('https://cocl.us/Geospatial_data', header=0)
geo_coord_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
geo_coord_df.shape

(103, 3)

In [17]:
geo_coord_df.sort_values(by='Postal Code', ascending=True)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


<b>Merging the two dataframes</b>

In [31]:
Toronto_with_geo_coord_df = pd.merge(Toronto_df, geo_coord_df, on='Postal Code')
Toronto_with_geo_coord_df.rename(columns={'Postal Code' : 'PostalCode'}, inplace=True)
Toronto_with_geo_coord_df.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


<b>Verifying shape of merged dataframe</b>

In [20]:
Toronto_with_geo_coord_df.shape

(103, 5)